In [1]:
import numpy as np
import pandas as pd
import gc

### read data

In [2]:
data = pd.read_pickle('./data.pkl')
data = data[[
    'date_block_num',
    'shop_id',
    'item_id',
    'item_cnt_month',
    #'city_code',
    'item_category_id',
    'type_code',
    'subtype_code',
    'item_cnt_month_lag_1',
    'item_cnt_month_lag_2',
    'item_cnt_month_lag_3',
    'item_cnt_month_lag_6',
    'item_cnt_month_lag_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_2',
    'date_item_avg_item_cnt_lag_3',
    'date_item_avg_item_cnt_lag_6',
    #'date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1',
    'date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_3',
    'date_shop_avg_item_cnt_lag_6',
    #'date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    #'date_shop_type_avg_item_cnt_lag_1',
    #'date_shop_subtype_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    #'date_type_avg_item_cnt_lag_1',
    #'date_subtype_avg_item_cnt_lag_1',
    'delta_price_lag',
    'month',
    #'days',
    'item_shop_last_sale',
    'item_last_sale',
    'item_shop_first_sale',
    'item_first_sale',
]]

In [3]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [4]:
del data
gc.collect();

### write submision

In [5]:
import csv
def writeFile(predic):
    with open('output.csv', 'w', newline='') as csvfile:
      # 建立 CSV 檔寫入器
      writer = csv.writer(csvfile)

      # 寫入一列資料
      writer.writerow(['ID','item_cnt_month'])
      index = 0  
      for data in predic:
          writer.writerow([str(index),str(data)])
          index+=1


### LinearRegression

In [6]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(X_train, Y_train)

to_be_predicted = np.array(X_test)
predicted_sales = lm.predict(to_be_predicted)

print(predicted_sales)

C:\Users\Lisa\Anaconda3\envs\tensorflow\lib\site-packages\numpy\core\_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


[ 0.45105982  0.35654303  1.1428175  ... -0.00669518  0.10681294
  0.03615908]


In [7]:
writeFile(list(np.maximum(np.array(predicted_sales+0.11), 0)))

In [8]:
print(predicted_sales+0.11)

[0.56105983 0.46654302 1.2528175  ... 0.10330482 0.21681294 0.14615908]


### Keras dense

In [9]:
from keras.models import Model
from keras.layers import *
from keras.models import Sequential

Using TensorFlow backend.


In [11]:
model = Sequential()
model.add(Dense(32, input_shape=(30,), kernel_initializer='normal', activation='relu'))
model.add(Dense(128, kernel_initializer='normal', activation='softmax'))
model.add(Dense(32,kernel_initializer='normal', activation='relu'))
model.add(Dense(16,kernel_initializer='normal', activation='softmax'))
model.add(Dense(1,kernel_initializer='normal', activation='relu'))

model.compile(loss='mse', optimizer='adam') 

train_history = model.fit(x=X_train, y=Y_train, validation_data=([X_valid,Y_valid]), epochs=30, batch_size=1000, verbose=2)  

Train on 6186922 samples, validate on 238172 samples
Epoch 1/10
 - 189s - loss: 1.4110 - val_loss: 1.2894
Epoch 2/10
 - 194s - loss: 1.4084 - val_loss: 1.2618
Epoch 3/10
 - 190s - loss: 1.2758 - val_loss: 1.1771
Epoch 4/10
 - 191s - loss: 1.2376 - val_loss: 1.2270
Epoch 5/10
 - 192s - loss: 1.1866 - val_loss: 1.0984
Epoch 6/10
 - 192s - loss: 1.1384 - val_loss: 1.0847
Epoch 7/10
 - 192s - loss: 1.1486 - val_loss: 1.0923
Epoch 8/10
 - 195s - loss: 1.1167 - val_loss: 1.1132
Epoch 9/10
 - 191s - loss: 1.0759 - val_loss: 1.1325
Epoch 10/10
 - 191s - loss: 1.0606 - val_loss: 1.0792


### Keras lstm

In [21]:
from keras import layers

model_lstm = Sequential()

model_lstm.add(layers.RepeatVector(3))
model_lstm.add(Dense(256,kernel_initializer='normal', activation='relu'))
model_lstm.add(LSTM(256, return_sequences=True))
model_lstm.add(Flatten())
model_lstm.add(Dense(256,kernel_initializer='normal', activation='relu'))
model_lstm.add(Dense(1,kernel_initializer='normal', activation='relu'))

model_lstm.compile(loss='mse', optimizer='adam') 

train_history = model_lstm.fit(x=np.array(X_train), y=np.array(Y_train), validation_data=([np.array(X_valid),np.array(Y_valid)]), epochs=10, batch_size=1000, verbose=2)  

Train on 6186922 samples, validate on 238172 samples
Epoch 1/10
 - 766s - loss: 1.4954 - val_loss: 1.3577
Epoch 2/10
 - 766s - loss: 1.4954 - val_loss: 1.3577
Epoch 3/10
 - 701s - loss: 1.4954 - val_loss: 1.3577
Epoch 4/10
 - 698s - loss: 1.4954 - val_loss: 1.3577
Epoch 5/10
 - 697s - loss: 1.4954 - val_loss: 1.3577
Epoch 6/10
 - 696s - loss: 1.4954 - val_loss: 1.3577
Epoch 7/10
 - 696s - loss: 1.4954 - val_loss: 1.3577
Epoch 8/10
 - 694s - loss: 1.4954 - val_loss: 1.3577
Epoch 9/10
 - 691s - loss: 1.4954 - val_loss: 1.3577
Epoch 10/10
 - 694s - loss: 1.4954 - val_loss: 1.3577


In [22]:
model_lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
repeat_vector_15 (RepeatVect (None, 3, 30)             0         
_________________________________________________________________
dense_32 (Dense)             (None, 3, 256)            7936      
_________________________________________________________________
lstm_14 (LSTM)               (None, 3, 256)            525312    
_________________________________________________________________
flatten_8 (Flatten)          (None, 768)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 256)               196864    
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 257       
Total params: 730,369
Trainable params: 730,369
Non-trainable params: 0
_________________________________________________________________
